In [1]:
import cv2 as cv2
import matplotlib.pyplot as plt
import numpy as np

In [36]:
import cv2
import numpy as np

# Загрузка видео
cap = cv2.VideoCapture('test.mp4')
if not cap.isOpened():
    print("Error opening video stream or file")

# Параметры для фона
background_frames = []
mean_background = None
threshold_value = 50  # Начальный порог для бинаризации
num_background_frames = 10

# Считываем первые несколько кадров для усредненного фона
for _ in range(num_background_frames):
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(cv2.resize(frame, (512, 512)), cv2.COLOR_BGR2GRAY)
    background_frames.append(frame_gray)

# Вычисляем средний фон и его среднюю яркость
mean_background = np.mean(background_frames, axis=0).astype(np.uint8)
mean_background = cv2.GaussianBlur(mean_background, (3, 3), 0)
background_brightness = np.mean(mean_background)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(cv2.resize(frame, (512, 512)), cv2.COLOR_BGR2GRAY)

    # Коррекция яркости текущего кадра
    frame_brightness = np.mean(frame_gray)
    brightness_correction_factor = background_brightness / (frame_brightness + 1e-5)
    frame_gray = cv2.convertScaleAbs(frame_gray, alpha=brightness_correction_factor)

    delta = cv2.absdiff(frame_gray, mean_background)

    _, mask = cv2.threshold(delta, threshold_value, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Убираем мелкие шумы с помощью морфологических операций
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)

    connectivity=4
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
    min_area = 3000
    filtered_mask = np.zeros_like(mask)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            filtered_mask[labels == i] = 255

    # Дополнительная морфология для сглаживания границ
    filtered_mask = cv2.morphologyEx(filtered_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    result = cv2.bitwise_and(frame_gray, frame_gray, mask=filtered_mask)

    cv2.imshow('Frame', result)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
